# TODO: 

-- README/Citations goes here

-- figure out 256px output (only does >= 512px right now)

-- integrate settings capture for posting in dalle2-results

-- probably fix the aesthetic loss function (aesthetic loss on cutouts)

## obligatory nvidia-smi

In [1]:
!nvidia-smi

Sun May 22 21:33:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## grab some stuff

In [3]:
!pip install git+https://github.com/nousr/DALLE2-pytorch.git # can probaly switch to lucidrains, we can always revert to a hash if a breaking change is introduced
!pip install git+https://github.com/nousr/deep-image-prior.git # pip-installable fork
!pip install madgrad # for default deep image prior generation

  Cloning https://github.com/nousr/DALLE2-pytorch.git to /tmp/pip-req-build-atrqb82v
  Running command git clone -q https://github.com/nousr/DALLE2-pytorch.git /tmp/pip-req-build-atrqb82v
     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 512 kB 64.5 MB/s 
     |████████████████████████████████| 1.4 MB 51.1 MB/s 
     |████████████████████████████████| 1.7 MB 52.6 MB/s 
     |████████████████████████████████| 46 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 66.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for dalle2-pytorch: filename=dalle2_pytorch-0.3.5-py3-none-any.whl size=1402867 sha256=aac118e409ccfe4e925d32b6ea3c1234f1ab79ebcac0c8086d6d562a044f0870
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ec9dwqv/wheels/9b/ec/9f/8b78188478a5a0ab8a24f23a5889a1dd754ccfb6b07cdb29b7
Successfully built dalle2-pytorch
  Cloning https://github.com/nousr/deep-image-prior.git to /tmp/pip-req-build-iumn4

## download models and style injection embedding

In [4]:
!wget https://huggingface.co/zenglishuci/conditioned-prior/resolve/main/vit-l-87k.pth # 100k checkpoint may have overfit a bit, use 87k for now
!wget https://github.com/LAION-AI/aesthetic-predictor/raw/main/vit_l_14_embeddings/rating9.npy

--2022-05-22 19:32:59--  https://huggingface.co/zenglishuci/conditioned-prior/resolve/main/vit-l-87k.pth
Resolving huggingface.co (huggingface.co)... 34.197.58.156, 18.214.24.217, 34.232.89.2, ...
Connecting to huggingface.co (huggingface.co)|34.197.58.156|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/82/5d/825dabd3e1765348e86eaa342efb982262672ddfc2273ec0d9ad7d589afd8587/da0e3ba64717705fc0439dfd5aa790a8c8da068a423fdfc75d9db487e27a2146?response-content-disposition=attachment%3B%20filename%3D%22vit-l-87k.pth%22 [following]
--2022-05-22 19:32:59--  https://cdn-lfs.huggingface.co/repos/82/5d/825dabd3e1765348e86eaa342efb982262672ddfc2273ec0d9ad7d589afd8587/da0e3ba64717705fc0439dfd5aa790a8c8da068a423fdfc75d9db487e27a2146?response-content-disposition=attachment%3B%20filename%3D%22vit-l-87k.pth%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 52.84.18.102, 52.84.18.121, 52.84.18.77, ...
Connecting to cdn-lfs.h

# Setup

In [1]:
from collections import namedtuple
from copy import deepcopy
import os
import io
import json
import gc
import math
import pickle
import random
import sys
import time

from einops import rearrange
import kornia.augmentation as K
from madgrad import MADGRAD
from resize_right import resize
import numpy as np
import requests
import torch
from torch import nn, optim
from torch.nn import functional as F
import torchvision.transforms.functional as TF
import torchvision.transforms as T
from tqdm.notebook import trange, tqdm

from dalle2_pytorch import DiffusionPrior, DiffusionPriorNetwork,OpenAIClipAdapter, DiffusionPriorTrainer

from deep_image_prior.models import get_hq_skip_net, get_non_offset_params, get_offset_params, skip
from deep_image_prior.utils import sr_utils

from clip import tokenize

# for resize right floor division
import warnings
warnings.simplefilter("ignore", UserWarning)


import torch
import torch.nn as nn
from os.path import expanduser  # pylint: disable=import-outside-toplevel
from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


## Helper functions

In [2]:
# ------------------------------------------------------------------------------
# Experimental Stuff
# ------------------------------------------------------------------------------
def inject_style(emb, style, weight=0.5):
  summation = (emb + weight*style)
  summation_norm = summation.norm()
  return summation/summation_norm

def get_aesthetic_model(clip_model="vit_l_14"):
    """load the aethetic model"""
    home = expanduser("~")
    cache_folder = home + "/.cache/emb_reader"
    path_to_model = cache_folder + "/sa_0_4_"+clip_model+"_linear.pth"
    if not os.path.exists(path_to_model):
        os.makedirs(cache_folder, exist_ok=True)
        url_model = (
            "https://github.com/LAION-AI/aesthetic-predictor/blob/main/sa_0_4_"+clip_model+"_linear.pth?raw=true"
        )
        urlretrieve(url_model, path_to_model)
    if clip_model == "vit_l_14":
        m = nn.Linear(768, 1)
    elif clip_model == "vit_b_32":
        m = nn.Linear(512, 1)
    else:
        raise ValueError()
    s = torch.load(path_to_model)
    m.load_state_dict(s)
    m.eval()
    return m

def aesthetic_loss_cutouts(cutouts, aesthetic_model, weight=0.05):
  """
  Perform aesthetic loss on the cutouts

  TODO: can probably done way better, just wanted to block this functionality in
  """
  aesthetic_losses = 0
  for cutout in cutouts:
    aesthetic_losses += aesthetic_model(cutout).item()
  
  # average out over all cutouts
  aesthetic_losses /= cutouts.shape[0]

  # clip if below zero
  if aesthetic_losses <= 0:
    aesthetic_losses = 1e-16

  return weight/aesthetic_losses
# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
# Usual clip-guidance functions
# ------------------------------------------------------------------------------

def seed_everything(seed: int):
    """
    from https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
    """
    import random, os
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True

class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)

replace_grad = ReplaceGrad.apply


def clamp(x, min_value, max_value):
    return max(min(x, max_value), min_value)


class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        # self.cut_pow = cut_pow
        self.augs = T.Compose([
            K.RandomHorizontalFlip(p=0.5),
            K.RandomAffine(degrees=15, translate=0.1, p=0.8, padding_mode='border', resample='bilinear'),
            K.RandomPerspective(0.4, p=0.7, resample='bilinear'),
            K.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.7),
            K.RandomGrayscale(p=0.15),
        ])

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        long_size, short_size = max(sideX, sideY), min(sideX, sideY)
        min_size = min(short_size, self.cut_size)
        pad_x, pad_y = long_size - sideX, long_size - sideY
        input_zero_padded = F.pad(input, (pad_x, pad_x, pad_y, pad_y), 'constant')
        input_mask = F.pad(torch.zeros_like(input), (pad_x, pad_x, pad_y, pad_y), 'constant', 1.)
        input_padded = input_zero_padded + input_mask * input.mean(dim=[2, 3], keepdim=True)
        cutouts = []
        for cn in range(self.cutn):
            if cn >= self.cutn - self.cutn // 4:
                size = long_size
            else:
                size = clamp(int(short_size * torch.zeros([]).normal_(mean=.8, std=.3)), min_size, long_size)
            # size = int(torch.rand([])**self.cut_pow * (short_size - min_size) + min_size)
            offsetx = torch.randint(min(0, sideX - size), abs(sideX - size) + 1, ()) + pad_x
            offsety = torch.randint(min(0, sideY - size), abs(sideY - size) + 1, ()) + pad_y
            cutout = input_padded[:, :, offsety:offsety + size, offsetx:offsetx + size]
            # cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
            cutouts.append(resize(cutout, out_shape=(self.cut_size, self.cut_size),by_convs=True, pad_mode='reflect'))
        return self.augs(torch.cat(cutouts))


class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))

    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()

class CaptureOutput:
    """Captures a layer's output activations using a forward hook."""

    def __init__(self, module):
        self.output = None
        self.handle = module.register_forward_hook(self)

    def __call__(self, module, input, output):
        self.output = output

    def __del__(self):
        self.handle.remove()

    def get_output(self):
        return self.output
# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
# Model loading 
# ------------------------------------------------------------------------------
def load_dip(input_depth, num_scales, offset_type, offset_groups, device):
    dip_net = get_hq_skip_net(
        input_depth,
        skip_n33d=192,
        skip_n33u=192,
        skip_n11=4,
        num_scales=num_scales,
        offset_type=offset_type,
        offset_groups=offset_groups
    ).to(device)

    return dip_net

def load_diffusion_model(dprior_path, device, clip_choice):

    loaded_obj = torch.load(str(dprior_path), map_location='cpu')
    
    if clip_choice == "vit_b_32":
      dim = 512
      clip_choice = "ViT-B/32"
    else:
      dim = 768
      clip_choice = "ViT-L/14"

    prior_network = DiffusionPriorNetwork(
        dim=dim,
        depth=12,
        dim_head=64,
        heads=12,
        normformer=True
    ).to(device)

    diffusion_prior = DiffusionPrior(
        net=prior_network,
        clip=OpenAIClipAdapter(clip_choice),
        image_embed_dim=dim,
        timesteps=1000,
        cond_drop_prob=0.1,
        loss_type="l2",
    ).to(device)


    diffusion_prior.load_state_dict(loaded_obj["model"], strict=True)

    diffusion_prior = DiffusionPriorTrainer(
                      diffusion_prior = diffusion_prior,
                      lr = 1.1e-4,
                      wd = 6.02e-2,
                      max_grad_norm = 0.5,
                      amp = False,
                  ).to(device)

    diffusion_prior.optimizer.load_state_dict(loaded_obj['optimizer'])
    diffusion_prior.scaler.load_state_dict(loaded_obj['scaler'])

    return diffusion_prior
# ------------------------------------------------------------------------------

## Load static models

In [3]:
clip_choice = "vit_l_14"
aesthetic_model = get_aesthetic_model(clip_model=clip_choice).to(device)
print("loaded aesthetic model")
diffusion_prior = load_diffusion_model("vit-l-87k.pth", 'cuda', clip_choice).eval().requires_grad_(False)
print("loaded model")

# load the style embedding
aesthetic_embedding = torch.from_numpy(np.load("rating9.npy")).to(device)

loaded aesthetic model
loaded model


# ⚙ Settings 

In [6]:
# hidden settings for nerds
offset_type = 'none'
num_scales = 7
lr = 1e-3
offset_lr_fac = 1.0
lr_decay= 0.995
input_depth = 32
optimizer_type = 'MADGRAD'  # Adam, MADGRAD
clip_size = 224 # can probably stay like this unless we train a ViT-L/14@336

# @markdown ### Generation Settings
# @markdown
cutn = 20 #@param {type:"integer"}
size = [512, 512] #@param ["[512, 512]", "[768, 512]", "[512, 768]"] {type:"raw", allow-input: true}
iterations = 500 #@param {type:"integer"}
display_freq = 25 #@param {type:"integer"}
disable_deformable_convolutions = False #@param {type:"boolean"}
seed = 0 #@param {type: "integer"}

# TODO: add seeding

# @markdown ---
# @markdown ### Experimental Settings
# @markdown
use_aesthetic_loss = True #@param {type:"boolean"}
aesthetic_loss_weight = 0.05 #@param {type:"slider", min:0, max:0.1, step:0.01}
inject_style_embedding = True #@param {type:"boolean"}
style_injection_weight = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}

# @markdown ---
# @markdown ### Additonal Settings
# @markdown
input_scale = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
input_noise_strength = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
param_noise_strength = 0.05 #@param {type:"slider", min:0, max:1, step:0.01}

# instantiate variables from specified settings
normalize = T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                        std=[0.26862954, 0.26130258, 0.27577711])
sideX, sideY = size

# set seeds
seed_everything(seed)

# disable deform convs
offset_groups = 0 if disable_deformable_convolutions else 4

# 🎨 Actually do the run!

In [ ]:
#@title Generate
prompt = "A beautiful painting of sailboats, in the style of greg rutkowski" #@param {type:"string"}
tokenized_text = tokenize(prompt).to(device)

# Initialize DIP skip network 
net = load_dip(input_depth=input_depth,
               num_scales=num_scales, 
               offset_type=offset_type, 
               offset_groups=offset_groups, 
               device=device)
# init the cutouts
make_cutouts = MakeCutouts(clip_size, cutn)

# Initialize input noise
net_input = torch.randn([1, input_depth, sideY, sideX], device=device)


noise = torch.randn((1,768))
t = torch.linspace(1, 0, 1000 + 1)[:-1]

score = 0
target_embed = diffusion_prior.sample(tokenized_text, num_samples_per_batch=2)
score = aesthetic_model(target_embed).item()
print(f"\n>> Aesthetic without injecting style: {score}")

if inject_style_embedding == True:
  target_embed = inject_style(target_embed, aesthetic_embedding, weight=style_injection_weight)
  score = aesthetic_model(target_embed).item()
  print(f">> Aesthetic score after injecting style: {score}\n")

prompts = [Prompt(target_embed)]

params = [{'params': get_non_offset_params(net), 'lr': lr},
          {'params': get_offset_params(net), 'lr': lr * offset_lr_fac}]

if optimizer_type == 'Adam':
    opt = optim.Adam(params, lr)
elif optimizer_type == 'MADGRAD':
    opt = MADGRAD(params, lr, momentum=0.9)

scaler = torch.cuda.amp.GradScaler()

image = None

itt = 0
try:
    for _ in trange(iterations, leave=True, position=0, colour='#DD6E42'):
        opt.zero_grad(set_to_none=True)

        noise_ramp = 1 - min(1, itt / iterations)
        net_input_noised = net_input

        if input_noise_strength:
            phi = min(1, noise_ramp * input_noise_strength) * math.pi / 2
            noise = torch.randn_like(net_input)
            net_input_noised = net_input * math.cos(phi) + noise * math.sin(phi)

        with torch.cuda.amp.autocast():
            out = net(net_input_noised * input_scale).float()

        losses = []
        cutouts = normalize(make_cutouts(out))
        
        with torch.cuda.amp.autocast(False):
            image_embeds = diffusion_prior.diffusion_prior.clip.clip.encode_image(cutouts).float()
        
        for prompt in prompts:

            if use_aesthetic_loss:
              # print(f"Aesthetic loss of cutouts: {aesthetics}")
              aesthetics = aesthetic_loss_cutouts(image_embeds, aesthetic_model=aesthetic_model, weight=aesthetic_loss_weight)
              losses.append(prompt(image_embeds) + aesthetics)
            else:
              losses.append(prompt(image_embeds))

        loss = sum(losses, out.new_zeros([]))
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        if param_noise_strength:
            with torch.no_grad():
                noise_ramp = 1 - min(1, itt / iterations)
                for group in opt.param_groups:
                    for param in group['params']:
                        param += torch.randn_like(param) * group['lr'] * param_noise_strength * noise_ramp

        itt += 1

        if itt % display_freq == 0 :
            with torch.inference_mode():
                image = TF.to_pil_image(out[0].clamp(0, 1))
                if itt % display_freq == 0:
                    losses_str = ', '.join([f'{loss.item():g}' for loss in losses])
                    tqdm.write(f'i: {itt}, loss: {loss.item():g}, losses: {losses_str}')
                    image.save(f'out_{itt:05}.png')
                    display(image, display_id=1)
        for group in opt.param_groups:
            group['lr'] = lr_decay * group['lr']

except KeyboardInterrupt:
    torch.cuda.empty_cache()
    gc.collect()
    if net is not None:
      del net
    pass

del net